In [2]:
import numpy as np
import os, sys
import mne
# need to install pyqt5/6, pip install pyqt6

# the trick to have interactive plots
%matplotlib qt 

In [3]:
eeg = np.load('/Users/nishant/Dropbox/Sinha/Lab/Research/ieeg_atlas_harmonization/data/hup/derivatives/bipolar/sub-RID0031/interictal_eeg_bipolar_0.pkl', allow_pickle=True)


Index(['LA01', 'LA02', 'LA03', 'LA04', 'LA05', 'LA06', 'LA07', 'LA08', 'LA09',
       'LB01',
       ...
       'RH07', 'RH08', 'RH09', 'RI01', 'RI02', 'RI03', 'RI04', 'RI05', 'RI06',
       'RI07'],
      dtype='object', length=109)

In [9]:

# load data in or read edf files
key = '15'
e = int(key)-1
plot_pred = True
module = 'sparcnet-update'
add_1st = False
data = eeg.values
labels = eeg.columns
fs = 200
valid_index = np.all(~np.isnan(data),axis=1)
data[~valid_index,:] = 0

# use mne to plot the data
info = mne.create_info(ch_names=labels, sfreq=fs, ch_types=['eeg'] * len(labels))
raw = mne.io.RawArray(data.T, info)  # Note: data needs to be transposed
raw.plot(show=True)


TypeError: ch_names must be an instance of list, tuple, or int, got <class 'pandas.core.indexes.base.Index'> instead.

In [ ]:

# create mne object, filter and set montage
info = mne.create_info(ch_names = old_ch_labels, sfreq = fs, ch_types = 'eeg')
raw = mne.io.RawArray(data.T*1e-6, info)
# raw.resample(256)
raw.filter(1., 40., method='iir',iir_params=dict(order=4, ftype='butter', output='sos'))
raw.notch_filter(freqs=[60],method='iir',iir_params=dict(order=4, ftype='butter', output='sos'))
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw = mne.set_bipolar_reference(raw,['Fp1','F7','T3','T5','Fp2','F8','T4','T6',
                    'Fp1','F3','C3','P3','Fp2','F4','C4','P4','Fz'],['F7','T3','T5','O1','F8','T4','T6','O2',
                    'F3','C3','P3','O1','F4','C4','P4','O1','Cz'])

# get start and duration of each event
sz_range = extract_seiz_ranges(labels)
sz_starts = np.array([a[0]/fs for a in sz_range])
sz_dura = np.array([(a[1]-a[0])/fs for a in sz_range])
sz_text = ['SS_Sz']*len(sz_starts)
# set annotations object
annotations = mne.Annotations(onset=np.concatenate([sz_starts]), duration=np.concatenate([sz_dura]), description=sz_text)


# Add annotations to raw object
raw.set_annotations(annotations)

raw.plot(title=key)